# Задание

Создание модели UNet.
Используя ранее реализованные блоки, соберите полную модель UNet,
соответствующую той, что была приведена на лекции.
Определите encoder, bottleneck и decoder части сети, добавьте skip
connections. Учтите, что для skipping connections тензоры необходимо
обрезать до нужного размера.

В сети UNet содержится:
- 4 блока UNetBlock которые мы уже реализовали.
- бутылочное горлышко. Заметим, что оно почти такое же, как UNetBlock, за
исключением операции пулинга. Но это нам не помешает, так как наш
UNetBlock возвращает и значения до этой операции
- 4 блока апсемплинга UNetUpBlock, которые принимают как тензор, который
мы будем увеличивать, так и skip connection тензор из энкодера.
- Замыкающую операцию свертки с ядром 1 и двумя фильтрами.


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [6]:
def crop_tensor_to_match(tensor, target_tensor):
    """
    Функция для обрезания тензора до размера, который должен быть в 2 раза больше размера target_tensor.
    Аргументы:
    - tensor: исходный тензор, который нужно обрезать.
    - target_tensor: тензор, по размеру которого мы ориентируемся.

    Задача: обрезать tensor так, чтобы его высота и ширина стали ровно в 2 раза больше, чем у target_tensor.

    Пояснение по размерам тензоров:
    - tensor.size() возвращает кортеж с размерами тензора в формате (batch_size, channels, height, width).
    - target_tensor.size() — аналогично, размеры целевого тензора.

    Переменные:
    - target_x = target_tensor.size()[3] * 2
      — ширина (axis 3) target_tensor, умноженная на 2.
    - target_y = target_tensor.size()[2] * 2
      — высота (axis 2) target_tensor, умноженная на 2.
    - diffY = tensor.size()[2] - target_y
      — разница между высотой исходного тензора и нужной высотой (в 2 раза больше целевого).
    - diffX = tensor.size()[3] - target_x
      — разница между шириной исходного тензора и нужной шириной.

    Обрезание:
    - Используем срезы по высоте и ширине:
      tensor[:, :, start_y:end_y, start_x:end_x]
      где start_y = diffY // 2 — сдвиг с верхнего края для центрирования обрезки,
      end_y = start_y + target_y — конечная точка по высоте,
      аналогично по ширине.

    Возвращаемый результат:
    - cropped_tensor — тензор, обрезанный по центру, с размерами (batch_size, channels, target_y, target_x),
      то есть в 2 раза больше, чем размеры target_tensor.
    """
    target_x = target_tensor.size()[3]*2  # вычисляем нужную ширину: в 2 раза больше, чем у target_tensor
    target_y = target_tensor.size()[2]*2  # вычисляем нужную высоту: в 2 раза больше, чем у target_tensor
    diffY = tensor.size()[2] - target_y   # на сколько исходный тензор больше по высоте
    diffX = tensor.size()[3] - target_x   # на сколько исходный тензор больше по ширине
    # обрезаем тензор по центру, сдвигаясь на половину излишка сверху и слева
    cropped_tensor = tensor[:, :, diffY // 2: diffY // 2 + target_y, diffX // 2: diffX // 2 + target_x]
    return cropped_tensor  # возвращаем обрезанный тензор


In [7]:
class UNetBlock(nn.Module):  # Определяем блок U-Net, наследуем от nn.Module (базовый класс нейросетей в PyTorch)
    def __init__(self, in_channels, out_channels):
        super(UNetBlock, self).__init__()  # Инициализация базового класса

        # Первый сверточный слой:
        # in_channels — число входных каналов (например, 3 для RGB),
        # out_channels — число выходных каналов (фильтров),
        # kernel_size=3 — размер фильтра 3x3,
        # padding=0 — без добавления пикселей на границе, поэтому размер изображения после свертки уменьшится
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=0)

        # Функция активации ReLU:
        # inplace=True — изменяет данные на месте без создания нового объекта (экономит память)
        self.relu = nn.ReLU(inplace=True)

        # Второй сверточный слой:
        # in_channels и out_channels равны out_channels первого слоя,
        # kernel_size и padding аналогичны первому,
        # каждый сверточный слой — отдельный слой с собственными весами
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=0)

        # Операция максимального объединения (max pooling):
        # kernel_size=2 — берет максимум из каждой области 2x2,
        # stride=2 — сдвигается на 2 пикселя, уменьшая размер изображения вдвое по ширине и высоте
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        # Прямой проход данных через блок

        x = self.conv1(x)  # Проходим через первую свертку
        x = self.relu(x)   # Применяем ReLU — обнуляем отрицательные значения
        x = self.conv2(x)  # Проходим через вторую свертку
        x = self.relu(x)   # Снова ReLU

        pooled = self.pool(x)  # Применяем max pooling, уменьшая размер

        # Возвращаем два значения:
        # x — результат после двух сверток (будет использоваться для пропуска (skip connection) в U-Net),
        # pooled — уменьшенная версия для следующего уровня сети
        return x, pooled

In [8]:
class UNetUpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetUpBlock, self).__init__()
        # nn.ConvTranspose2d - транспонированная свертка, используется для увеличения пространственных размеров (upsampling).
        # Параметры:
        # in_channels - количество входных каналов,
        # out_channels - количество выходных каналов,
        # kernel_size=2 - размер ядра свертки 2x2,
        # stride=2 - шаг 2 для удвоения размера входа (например, 64x64 -> 128x128).
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

        # nn.Conv2d - обычная 2D-свертка для извлечения признаков.
        # Параметры:
        # in_channels - количество входных каналов,
        # out_channels - количество выходных каналов,
        # kernel_size=3 - размер ядра 3x3,
        # padding=0 - без заполнения, поэтому размер выходного тензора уменьшается.
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=0)

        # nn.ReLU - функция активации ReLU (Rectified Linear Unit),
        # inplace=True означает, что операция делается "на месте" для экономии памяти.
        self.relu = nn.ReLU(inplace=True)

        # Второй сверточный слой с теми же параметрами, для более глубокого извлечения признаков.
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=0)

    def forward(self, x, skip_connection):
        # x - входной тензор (например, из предыдущего слоя U-Net, с уменьшенным пространственным размером).
        # skip_connection - тензор из соответствующего слоя вниз по U-Net, для объединения деталей.

        # Применяем транспонированную свертку для увеличения размера изображения (upsampling).
        x = self.upconv(x)

        # Объединяем (конкатенируем) по канальному измерению (dim=1)
        # upsampled x и skip_connection, чтобы сохранить детали из ранних слоев.
        x = torch.cat([x, skip_connection], dim=1)

        # Пропускаем результат через первый сверточный слой для извлечения признаков.
        x = self.conv1(x)

        # Применяем активацию ReLU для нелинейности.
        x = self.relu(x)

        # Пропускаем через второй сверточный слой для более детальной обработки.
        x = self.conv2(x)

        # Снова ReLU активация.
        x = self.relu(x)

        # Возвращаем обработанный тензор.
        return x


In [9]:

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        # Инициализация блоков энкодера (сжатия)
        # Каждый блок уменьшает размерность, увеличивает число каналов
        self.enc1 = UNetBlock(1, 64)    # Вход 1 канал (например, grayscale), выход 64 канала
        self.enc2 = UNetBlock(64, 128)  # Принимает 64 канала, отдаёт 128
        self.enc3 = UNetBlock(128, 256) # Принимает 128, отдаёт 256
        self.enc4 = UNetBlock(256, 512) # Принимает 256, отдаёт 512

        # Бутылочное горлышко — самый глубокий слой с максимальным числом каналов
        self.bottleneck = UNetBlock(512, 1024)  # 512 -> 1024 каналов

        # Блоки декодера (расширения) — увеличивают размер изображения, уменьшают число каналов
        self.up4 = UNetUpBlock(1024, 512)  # 1024 входных каналов, 512 выходных
        self.up3 = UNetUpBlock(512, 256)   # 512 -> 256
        self.up2 = UNetUpBlock(256, 128)   # 256 -> 128
        self.up1 = UNetUpBlock(128, 64)    # 128 -> 64

        # Финальный сверточный слой, kernel_size=1 — свертка 1x1 для уменьшения каналов до 2
        # Обычно для сегментации с 2 классами (фон и объект)
        self.final_conv = nn.Conv2d(64, 2, kernel_size=1)

    def forward(self, x):
        # Проход через энкодер:
        # Каждый блок возвращает два значения: output (xN) и pooled output (pN)
        x1, p1 = self.enc1.forward(x)  # Первый блок: вход x, выход x1, pooled p1
        x2, p2 = self.enc2.forward(p1) # Второй блок: вход p1, выход x2, pooled p2
        x3, p3 = self.enc3.forward(p2)
        x4, p4 = self.enc4.forward(p3)

        # Бутылочное горлышко:
        # Нужен только первый выход (не pooled), тк это центр сети
        x5, _ = self.bottleneck.forward(p4)

        # Декодер:
        # Здесь нужно соединить слои пропуска (skip connections) из энкодера с декодером
        # Для этого обрезаем тензор x4 по размеру x5 (с помощью crop_tensor_to_match)
        x4_skip = crop_tensor_to_match(x4, x5)
        x = self.up4.forward(x5, x4_skip)  # Объединяем bottleneck и skip connection x4

        x3_skip = crop_tensor_to_match(x3, x)  # Аналогично с x3
        x = self.up3.forward(x, x3_skip)

        x2_skip = crop_tensor_to_match(x2, x)
        x = self.up2.forward(x, x2_skip)

        x1_skip = crop_tensor_to_match(x1, x)
        x = self.up1.forward(x, x1_skip)

        # Финальный сверточный слой с ядром 1 для получения окончательного результата сегментации
        x = self.final_conv(x)

        return x


In [10]:

model = UNet()
# Создаем экземпляр модели UNet с заранее определённой архитектурой

input_tensor = torch.randn(1, 1, 572, 572)
# Создаем входной тензор:
# 1 - batch size (один пример за раз)
# 1 - количество каналов (например, grayscale изображение)
# 572 x 572 - размер изображения (ширина и высота)
# Значения случайные, с нормальным распределением (randn)

output = model(input_tensor)
# Прогоняем входной тензор через модель (прямой проход)
# Результат — тензор выхода модели

print(output.shape)
# Печатаем размер выходного тензора
# Ожидается, что размер будет примерно (1, 2, H_out, W_out)
# где 2 — количество каналов на выходе (например, классы сегментации),
# H_out и W_out зависят от архитектуры и паддингов свёрток.


torch.Size([1, 2, 388, 388])
